<a href="https://colab.research.google.com/github/HunewJeong/HunewJeong/blob/main/MG_Server_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn transformers torch pyngrok nest_asyncio

In [ ]:
!ngrok authtoken 2m9QYS2jLHM8i6Wxl9mMNEYr69Q_25mFkRo1HKwKP4scpfh4v

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
### kill all ngrok servers running
!kill $(pgrep ngrok)

In [ ]:
### check if the port 8000 is used; blank output means we're good to go
!lsof -i :8000

In [ ]:
### if lsof -i :8000 shows some output place the PID from the output (4 or 5 digits) in the hashtags
!kill -9 #####

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import FileResponse
from transformers import pipeline
import os
import torch
import nest_asyncio
from pyngrok import ngrok
import threading
import uvicorn
import soundfile as sf
import numpy as np
from scipy.io.wavfile import write


# Apply nest_asyncio to avoid event loop conflicts in Colab
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Setup the text-to-audio model (e.g., facebook/musicgen-melody)
model = "facebook/musicgen-melody"
device = 0 if torch.cuda.is_available() else -1

print(f"Setting up the pipeline with device: {device}...")
audio_pipeline = pipeline("text-to-audio", model=model, device=device)

# Create output directory for saving generated audio
output_dir = os.path.abspath("/content/generated_audio")
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory created at: {output_dir}")

# Define request model for audio generation
class Prompt(BaseModel):
    prompt: str

# Define FastAPI route for generating audio
from scipy.io.wavfile import write
import numpy as np

@app.post("/generate-audio")
async def generate_audio(prompt: Prompt):
    try:
        print(f"Received prompt: {prompt.prompt}")

        # Generate audio using the model
        with torch.no_grad():
            output = audio_pipeline(prompt.prompt)

        # Check and print the model's output
        print("Model output:", output)

        # Ensure output contains audio data
        if not output or "audio" not in output:
            raise ValueError("Model did not generate any audio.")

        # Extract the audio tensor
        audio_tensor = output['audio']
        print(f"Audio tensor shape before flattening: {audio_tensor.shape}")

        # Flatten the audio data (convert from 3D to 1D)
        audio_flattened = audio_tensor.flatten()
        print(f"Audio tensor shape after flattening: {audio_flattened.shape}")

        # Convert to numpy and scale to int16 format
        audio_np = np.int16(audio_flattened * 32767)  # Assuming audio is in [-1, 1] range

        # Define the output file path
        output_file = os.path.join(output_dir, "generated_music.wav")

        # Save the audio using scipy.io.wavfile.write
        write(output_file, output['sampling_rate'], audio_np)

        # Return the generated .wav file as a download
        return FileResponse(output_file, media_type="audio/wav", filename="generated_music.wav")

    except Exception as e:
        print(f"Error during audio generation: {str(e)}")
        return {"error": str(e)}

# Save the audio file with the correct sample
# Function to run the FastAPI app on port 8000
def run_server():
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Start the FastAPI server in a separate thread
thread = threading.Thread(target=run_server)
thread.start()

# Expose the FastAPI app to the internet via ngrok
public_url = ngrok.connect(8000)
print(f"FastAPI server is running at: {public_url}")

Setting up the pipeline with device: 0...


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Output directory created at: /content/generated_audio


INFO:     Started server process [40673]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


FastAPI server is running at: NgrokTunnel: "https://fa76-34-34-19-16.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
import requests

url = "https://fa76-34-34-19-16.ngrok-free.app/generate-audio"
payload = {
    "prompt": "Generate a happy melody"
}

response = requests.post(url, json=payload)

# Save the generated audio to a local file
with open("generated_music.wav", "wb") as f:
    f.write(response.content)

print("Audio saved as generated_music.wav")

Received prompt: Generate a happy melody
Model output: {'audio': array([[[-0.02366114, -0.02643369, -0.02257427, ..., -0.00355363,
         -0.01159055, -0.00451481]]], dtype=float32), 'sampling_rate': 32000}
Audio tensor shape before flattening: (1, 1, 957440)
Audio tensor shape after flattening: (957440,)
INFO:     34.34.19.16:0 - "POST /generate-audio HTTP/1.1" 200 OK
Audio saved as generated_music.wav
